In [1]:
def compute_overall_capex(
    cap_l, actualisation, equity_share, interest_rate, loan_years, plant_life
):
    """
    This function computes the initial capex investment from the reported share of CAPEX in a fuel MFSP.
    Standard hypothesis used to do so   => construction spans over years -1,0,1 with capital spent 30%,50%, 20%
                                        => production ramp-up: 30% year 1, 70% year 2, full load afterwards
                                        => Fuel density => 0.804 kg/L
                                        => "included" load factor ==> real rather that nominal output capacity
    :param cap_l: capex per liter (€2020/L)
    :param actualisation: actualisation/internal rate of return/hurdle rate => 10%
    :param equity_share: share of investment on investor own funds
    :param interest_rate: interest rate on loan
    :param loan_years: loan reimbursement period
    :param plant_life: plant expected life span
    :return: the initial CAPEx (€2020) required per ton per day investment
    """
    prod_cost_year_ton = cap_l / 0.804 * 1000 * 365

    prod_cost_life = prod_cost_year_ton * (
        0.3 / (1 + actualisation)
        + 0.7 / (1 + actualisation) ** 2
        + sum(1 / (1 + actualisation) ** i for i in range(3, plant_life + 1))
    )

    equity = equity_share * (0.3 * (1 + actualisation) + 0.5 + 0.2 / (1 + actualisation))
    # print(equity)
    debt = (1 - equity_share) * (
        sum(
            (1 / loan_years + interest_rate * (1 - i / loan_years)) / (1 + actualisation) ** i
            for i in range(0, loan_years)
        )
    )
    # print(debt)

    capx_per_ton_day = prod_cost_life / (equity + debt)
    per_liter = (
        capx_per_ton_day
        * (equity + debt)
        / (prod_cost_life / (prod_cost_year_ton * 0.804 / 1000 / 365))
    )

    print(per_liter)

    return capx_per_ton_day

In [2]:
def compute_energy_cost(feed_l, plant_efficiency, fuel_density):
    """
    Computes energy cost per MJ of intrant, depending on energy cost per liter of output, efficiecny and energy density of fuel produced
    """
    return feed_l / (fuel_density / plant_efficiency)

No need for additional function in the case of OPEX, input directly in €/L

In [3]:
compute_energy_cost(0.35, 0.6, 43)

0.004883720930232558

In [4]:
compute_overall_capex(0.340791, 0.1, 0.0, 0.08, 15, 25)

0.340791


1278312.5832469284